# 海洋对气候的影响（基于最近高温天气分析一波）
## weather data


**"""请求api：https://pro.openweathermap.org/data/2.5/forecast/hourly?lat={lat}&lon={lon}&appid={API key}
参数：lat，lon， apikey
转码器：http://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}
参数：cityname, apikey
mykey:
"""
**


In [1]:
import xlwings as xw
import requests as rq
import json
import pandas as pd
import time

In [2]:
key= "59d046440aa1e8adb751bb944ab422d4"

In [3]:
#读取excel文件
def get_city_names():
	file= r"C:\Users\HP\Desktop\海洋系统分析\距海城市.xlsx"
	wbook=xw.Book(file)
	wsheet=wbook.sheets["城市选择"]
	ran_citys=wsheet.range("A2:A17")
	ran_dis=wsheet.range("B2:B17")
	citys=[]
	dis=[]
	for num in range(0,16):
		name = ran_citys[num, 0].value
		distance=ran_dis[num,0].value
		citys.append(name)
		dis.append(distance)
	print("城市加载成功！！")
	return citys,dis


In [4]:
#获取经纬度
def get_con_alt(names):
    loc = list()
    print("正在获取经纬度信息.......")
    for name in names:
        url="http://api.openweathermap.org/geo/1.0/direct?q="+name+"&appid="+key
        response = rq.get(url)
        time.sleep(0.5)
        if(response.status_code!=200):
            print("请求出现错误， 请检查")
        content = response.json()#这里直接进行了反序列化
        # print(type(content))
        #处理json文件
        # content = json.load(response)
        # print(content)
        lat=content[0]["lat"]
        lon=content[0]["lon"]
        temp =[lat, lon]
        loc.append(temp)
        ######################################
        #####################################
        ####################################
#         break
    print("位置经纬度，获取成功！")
    return loc

In [5]:
def get_data(loc):
	lenght = len(loc)
	data_list=[]
	mykey= "59d046440aa1e8adb751bb944ab422d4"
	for i in range(lenght):
		url = "http://api.openweathermap.org/data/2.5/forecast?lat="+str(loc[i][0])+"&lon="+str(loc[i][1])+"&appid="+mykey
		#要在这里形成dataframe
		rs =rq.get(url)
		time.sleep(0.5)
		#直接进行了发序列化,这里rs的类型为字典
		rs = rs.json()
		#形成列表
		rs=[rs]
		#形成函数参数
		agr1=[]
		agr2=[]
		for key,value in rs[0].items():
			if isinstance(value, list):
				agr1.append(key)
			elif isinstance(value, dict):
				for j in rs[0][key].keys():
					l=[]
					l.append(key)
					l.append(j)
					agr2.append(l)
			else:
				agr2.append(key)
		df = pd.json_normalize(rs, agr1,agr2)
		data_list.append(df)
	return data_list

In [6]:
def deco_data(datas, distances):
	df_list=[]
	i =0
	for data in datas:
        #data is  a dataframe
		col_name= ['dt','city.name','city.name','weather','main.temp','main.pressure','main.humidity','wind.speed','wind.deg']
		df2=pd.DataFrame(data=data, columns=col_name)
		#进行每一列数据的修改，apply函数， dt列, 添加距离列
		df2["main.temp"] = df2["main.temp"]-273.15
		df2['dt'] = df2['dt'].apply(datetime.datetime.fromtimestamp)
		df2["dis"]=distances[i]
		i=i+1
		df_list.append(df2)
	return df_list

In [7]:
def save_to_local(dfs, city_names):
	i= 0
	for df in dfs:
		#查看数据形状
		print("数据形状如下！！！")
		print(df.shape)
		#将文件保存到本地
		#这里需要知道city_names,和距离列
		df.to_csv(city_names[i]+".csv")
		i=i+1
	print("数据已经全部加载到了本地，下面开始分析吧！！")

In [ ]:
city_names, dis =get_city_names()
loc = get_con_alt(city_names)
data_list = get_data(loc)
dfs = deco_data(data_list)
save_to_local(dfs,city_names)

城市加载成功！！
正在获取经纬度信息.......
位置经纬度，获取成功！


## 进行数据探索了！！！

In [2]:
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import sklearn.svm as svm
import numpy as np

In [3]:
### 进行数据分析了
##加载本地数据，16个城市
##第一组
path="D:\\100 of 100"
shantou= pd.read_csv(path+"\\"+"汕头.csv")
chaonan=pd.read_csv(path+"\\"+"朝南.csv")
puning=pd.read_csv(path+"\\"+"普宁.csv")
jiedong=pd.read_csv(path+"\\"+"揭东.csv")
##第二组
zhaozhou=pd.read_csv(path+"\\"+"潮州.csv")
fenshun=pd.read_csv(path+"\\"+"丰顺.csv")
beidou=pd.read_csv(path+"\\"+"北斗.csv")
xiandong=pd.read_csv(path+"\\"+"仙洞.csv")
##第三组
wuhau=pd.read_csv(path+"\\"+"五华.csv")
meizhou=pd.read_csv(path+"\\"+"梅州.csv")
zhangzhou=pd.read_csv(path+"\\"+"漳州.csv")
longyan=pd.read_csv(path+"\\"+"龙岩.csv")
##第四组
heping=pd.read_csv(path+"\\"+"和平.csv")
dingnan=pd.read_csv(path+"\\"+"定南.csv")
shaoguan=pd.read_csv(path+"\\"+"韶关.csv")
ganzhou=pd.read_csv(path+"\\"+"赣州.csv")

# print(shantou)

    Unnamed: 0                   dt city.name  \
0            0  2022-07-23 17:00:00   Longyan   
1            1  2022-07-23 20:00:00   Longyan   
2            2  2022-07-23 23:00:00   Longyan   
3            3  2022-07-24 02:00:00   Longyan   
4            4  2022-07-24 05:00:00   Longyan   
5            5  2022-07-24 08:00:00   Longyan   
6            6  2022-07-24 11:00:00   Longyan   
7            7  2022-07-24 14:00:00   Longyan   
8            8  2022-07-24 17:00:00   Longyan   
9            9  2022-07-24 20:00:00   Longyan   
10          10  2022-07-24 23:00:00   Longyan   
11          11  2022-07-25 02:00:00   Longyan   
12          12  2022-07-25 05:00:00   Longyan   
13          13  2022-07-25 08:00:00   Longyan   
14          14  2022-07-25 11:00:00   Longyan   
15          15  2022-07-25 14:00:00   Longyan   
16          16  2022-07-25 17:00:00   Longyan   
17          17  2022-07-25 20:00:00   Longyan   
18          18  2022-07-25 23:00:00   Longyan   
19          19  2022

In [ ]:
#选择第一个最近的城市
y1=shantou['main.temp']
x1=shantou['dt']
fig, ax=plt.subplot()
plt.xticks(rotation=70)
hours=mdates.DateFormatter('%H:%M')
ax.xaxis.set_major_formatters(hours)
ax.plot(x1, y1, 'r')

In [ ]:
#最高气温与沿海距离之间的关系
#选择第一组和第二组城市进行对比
y2_close=shantou['main.temp']
x2_close=shantou['dt']
y3_close=chaonan['main.temp']
x3_close=chaonan['dt']
y4_close=puning['main.temp']
x4_close=puning['dt']
y5_close=jiedong['main.temp']
x5_close=jiedong['dt']

y2_distance=heping['main.temp']
x2_distance=heping['dt']
y3_distance=dingnan['main.temp']
x3_distance=dingnan['dt']
y4_distance=shaoguan['main.temp']
x4_distance=shaoguan['dt']
y5_distance=ganzhou['main.temp']
x5_distance=ganzhou['dt']

fig, ax=plt.subplot()
plt.xticks(rotation=70)
hours=mdates.DateFormatter('%H:%M')
plt.plot(x2_close,y2_close,'r',x3_close,y3_close,'r', x4_close,y4_close,'r',x5_close,y5_close,'r')
plt.plot(x2_distance,y2_distance,'g',x3_distance,y3_distance,'g', x4_distance,y4_distance,'g',x5_distance,y5_distance,'g')


In [ ]:
#最低气温与沿海距离之间的关系
#采用SVR算法找出临界点
dis=[
    shantou['dis'][0], chaonan['dis'][0], puning['dis'][0],jiedong['dis'][0],
     zhangzhou['dis'][0],fenshun['dis'][0],beidou['dis'][0],xiandong['dis'][0],
     wuhau['dis'][0],meizhou['dis'][0],zhangzhou['dis'][0],longyan['dis'][0],
     heping['dis'][0],dingnan['dis'][0],shaoguan['dis'][0],ganzhou['dis'][0]
    ]
temp_max=[
	shantou['main.temp'].max(), chaonan['main.temp'].max(), puning['main.temp'].max(),jiedong['main.temp'].max(),
     zhangzhou['main.temp'].max(),fenshun['main.temp'].max(),beidou['main.temp'].max(),xiandong['main.temp'].max(),
     wuhau['main.temp'].max(),meizhou['main.temp'].max(),zhangzhou['main.temp'].max(),longyan['main.temp'].max(),
     heping['main.temp'].max(),dingnan['main.temp'].max(),shaoguan['main.temp'].max(),ganzhou['main.temp'].max()
	]
temp_min=[
	shantou['main.temp'].min(), chaonan['main.temp'].min(), puning['main.temp'].min(),jiedong['main.temp'].min(),
     zhangzhou['main.temp'].min(),fenshun['main.temp'].min(),beidou['main.temp'].min(),xiandong['main.temp'].min(),
     wuhau['main.temp'].min(),meizhou['main.temp'].min(),zhangzhou['main.temp'].min(),longyan['main.temp'].min(),
     heping['main.temp'].min(),dingnan['main.temp'].min(),shaoguan['main.temp'].min(),ganzhou['main.temp'].min()
	]

In [ ]:
#画图
plt.plot(dis,temp_max,'ro')
#SVR算法
svr_line1=svm.SVR(kernel='linear',C=1e3)
svr_line2=svm.SVR(kernel='linear',C=1e3)
svr_line1.fit(x2_close, y2_close)
svr_line2.fit(x3_close, y3_close)
xp1=np.arange(10,100,10).reshape(9,1)
xp2=np.arange(50,400,50).reshape(7,1)
yp1=svr_line1.predict(xp1)
yp2=svr_line2.predict(xp2)

plt.plot(xp1,yp1,c='b',label="Strong effect")
plt.plot(xp2,yp2, c='r', label="light effect")
plt.axis(0, 400, 27,32)
plt.scatter(x,y,c='k', label='data')

In [ ]:
#分析湿度与距离之间的关系
y6_close=shantou['main.humidity']
x6_close=shantou['dt']
y7_close=chaonan['main.humidity']
x7_close=chaonan['dt']
y8_close=puning['main.humidity']
x8_close=puning['dt']
y9_close=jiedong['main.humidity']
x9_close=jiedong['dt']

# zhaozhou=pd.read_csv()
# fenshun=pd.read_csv()
# beidou=pd.read_csv()
# xiandong=pd.read_csv()
# ##第三组
# wuhau=pd.read_csv()
# meizhou=pd.read_csv()
# zhangzhou=pd.read_csv()
# longyan=pd.read_csv()
# ##第四组
# heping=pd.read_csv()
# dingnan=pd.read_csv()
# shaoguan=pd.read_csv()
# ganzhou=pd.read_csv()
y6_distance=heping['main.humidity']
x6_distance=heping['dt']
y7_distance=dingnan['main.humidity']
x7_distance=dingnan['dt']
y8_distance=shaoguan['main.humidity']
x8_distance=shaoguan['dt']
y9_distance=ganzhou['main.humidity']
x9_distance=ganzhou['dt']

fig, ax=plt.subplot()
plt.xticks(rotation=70)
hours=mdates.DateFormatter('%H:%M')
plt.plot(x6_close,y6_close,'r',x7_close,y7_close,'r', x8_close,y8_close,'r',x9_close,y9_close,'r')
plt.plot(x6_distance,y6_distance,'g',x7_distance,y7_distance,'g', x8_distance,y8_distance,'g',x9_distance,y9_distance,'g')


In [ ]:
#湿度最小值
hum_min = [
	shantou['main.humidity'].min(), chaonan['main.humidity'].min(), puning['main.humidity'].min(),jiedong['main.humidity'].min(),
     zhangzhou['main.humidity'].min(),fenshun['main.humidity'].min(),beidou['main.humidity'].min(),xiandong['main.humidity'].min(),
     wuhau['main.humidity'].min(),meizhou['main.humidity'].min(),zhangzhou['main.humidity'].min(),longyan['main.humidity'].min(),
     heping['main.humidity'].min(),dingnan['main.humidity'].min(),shaoguan['main.humidity'].min(),ganzhou['main.humidity'].min()
]
#湿度最大值
hum_max = [
	shantou['main.humidity'].max(), chaonan['main.humidity'].max(), puning['main.humidity'].max(),jiedong['main.humidity'].max(),
     zhangzhou['main.humidity'].max(),fenshun['main.humidity'].max(),beidou['main.humidity'].max(),xiandong['main.humidity'].max(),
     wuhau['main.humidity'].max(),meizhou['main.humidity'].max(),zhangzhou['main.humidity'].max(),longyan['main.humidity'].max(),
     heping['main.humidity'].max(),dingnan['main.humidity'].max(),shaoguan['main.humidity'].max(),ganzhou['main.humidity'].max()
]
plt.plot(dis, hum_max, 'bo')

In [ ]:
#分析风向和风速与距离之间的关系
#采用极区图,这里要画10个图
# new_df = df
hist, bins=np.histogram(shantou['wind.deg'],8,(0,360))

def show_wind_reg(values, names, max_value):
    NUM=8
    theta=np.arange(0,2*np.pi, 2*np.pi/NUM)
    raddi=np.array(values)
    plt.axes([0.025,0.025,0.95,0.95], polar=True)
    colors=[(1-x/max_value, 1-x/max_value, 0.75) for i in raddi]
    plt.bar(theta, raddi, bottom=0.0, width=(2*np.pi/NUM), colors=colors)
    plt.title(label)
def show_wind_speed(df_citys):
    degs=np.arange(45,361,45)
    tmp=[]
    for deg in degs:
        tmp.append(df_citys[(df_citys['wind.speed']>(deg-46)) & (df_citys['wind.speed']<deg)],
                   df_citys['wind.speed'].mean())  
    return np.array(tmp)

#调用函数
# show_wind(hist,"shantou",max(hist))
# show_wind(show_wind_speed,"shantou")
#end